# QUANTUM CHEMISTRY SIMULATIONS IN PSI4 - LIVE CODING SESSION (STUDENT) #

## Learning outcomes

After completing these exercises, you will be able to

- explain the utility of electronic structure theory in modern chemistry
- explain the basic tenets of the Born-Oppenheimer approximation
- define the two main approximations used in calculating approximate solutions of the electronic Schrödinger equation
- explain how basis set size affect the accuracy of solutions to the electronic Schrödinger equation
- explain how the level of electron correlation treatment affects the accuracy of solutions to the electronic Schrödinger equation
- enumerate the information contained in a potential energy curve/surface
- demonstrate an understanding of the practical nuances of computational modeling

## <span style="color:blue"> Pre-lab activities </span>

<span style="color:blue">

1. Read sections I - II as well as section III.1 on specifying the molecular geometry in Z-matrix notation. Make sure to answer the quizzes in these sections.
2. In one sentence, describe what electronic structure theory is and provide two examples of its utility.
3. Provide a concise description of the Born-Oppenheimer approximation and provide the underlying justification for its application.
4. In one sentence, describe the term LCAO.
5. Consider a calculation for carbon monoxide at the HF level of theory using a cc-pVQZ basis set. Determine the number of i) occupied MOs, 2) total MOs, and 3) unoccupied virtual MOs.
6. Provide a **short** description of how correlated electronic structure methods account for the instantaneous repulsion between electrons. 
</p>

# <span style="color:red"> For the Jupyter notebook to display correctly, make sure to exectue the cell below </span>

<span style="color:red">
You only need to do this once at the beginning unless you restart the Python kernel.
</span>

In [1]:
import os

quiz_fname = 'quantum_chemistry_simulations_in_psi4_questions.py'

if ( os.path.exists(quiz_fname) == False ) :
    print("Python file with questions does not exist ... will now attempt to download")
    !wget https://raw.githubusercontent.com/act-cms/quantum-chemistry-simulations-in-psi4/refs/heads/main/quizzes/quantum_chemistry_simulations_in_psi4_questions.py
else:
    print("Python file with questions already exists ... carry on")

try:
    import jupyterquiz
    print("Jupyterquiz is available and ready for use ... carry on")
except ImportError:
    print("Jupyterquiz is not installed or not accessible in this environment ... will now attempt to install")
    %pip install jupyterquiz

from jupyterquiz import display_quiz
from quantum_chemistry_simulations_in_psi4_questions import *

Python file with questions already exists ... carry on
Jupyterquiz is available and ready for use ... carry on


# Scope of this Jupyter notebook

Computational chemistry plays a crucial role in modern chemistry, serving as a bridge between theoretical concepts and experimental observations. For students studying physical chemistry, familiarity with computational methods is essential, as these tools allow chemists to simulate complex molecular interactions, predict reaction outcomes, visualize chemical processes at an atomic level, and back up experimental results. By leveraging computational techniques, researchers can explore systems that may be difficult or impossible to study experimentally, thus enhancing our understanding of chemical phenomena and accelerating the development of new materials and pharmaceuticals. 

The field of computational chemistry encompasses many different areas including the analysis of qualitative structure property relationships (QSPR), the use of AI in predicting the three-dimensional structure of a protein from its amino acid sequence (think Alphafold2 and the 2024 Nobel prize in chemistry), and electronic structure theory. The examples in this worksheet will serve as an introduction to the utility of electronic structure theory in predicting the properties of small molecules, and I hope you will develop a deeper appreciation for the value of electronic structure methods in aiding experimental chemists. Below, you will find a brief discussion of the key ideas and approximations used in electronic structure theory. For a more detailed discussion, please see an introductory textbook on electronic structure theory or talk to your instructor.

In section I.1, the Born-Oppenheimer approximation and the form of the electronic Schrödinger equation are discussed. Section I.2 presents the basic tenets of electronic structure theory. Section II provides an abbreciated list of acronyms and abbreviations of methods used by computational chemists, and the rest of the notebook (section III) is focused on examples that demonstrate concepts and applications of electronic structure theory to predict molecular properties and reaction energetics. Although we use the Psi4 electronic structure package in this notebook, there are many others (e.g. GAMESS, Q-Chem, Columbus, Gaussian, PySCF, ORCA etc.); each of these packages has its own strengths and challenges, and which one you use will depend on the specific problem you wish to solve. Lastly, I would be remiss not to mention that computational/theoretical chemists love using acronyms; I will try to keep the jargon and acronyms to a minimum, however, they are nearly impossible to avoid.

# I. Introduction to key concepts of quantum chemistry modelling 

## I.1. The Born-Oppenheimer approximation, atomic units, and the electronic Schrödinger equation

The Hamiltonian for a molecule with $N_e$ electrons (labeled by lower case letters $i$ and $j$) and $N_{nuc}$ nuclei (labeled by upper case letters $A$ and $B$) is
\begin{eqnarray}
\hat{H}_{\rm molecular} &=& \hat{T}_e + \hat{V}_{Ne} + \hat{V}_{ee} + \hat{V}_{NN} + \hat{T}_N \
&=& -\frac{\hbar^2}{2m_e}\sum_{i=1}^{N_e} \nabla^2_i -\frac{e^2}{4\pi\varepsilon_0}\sum_{A=1}^{N_{nuc}}\sum_{i=1}^{N_e}\frac{Z_A}{r_{iA}}+\frac{e^2}{4\pi\varepsilon_0}\sum_{i=1}^{N_e}\sum_{j>i}^{N_e}\frac{1}{r_{ij}} + \frac{e^2}{4\pi \varepsilon_0} \sum_{A=1}^{N_{nuc}} \sum_{B>A}^{N_{nuc}} \frac{Z_AZ_b}{R_{AB}} -\frac{\hbar^2}{2m_A}\sum_{A=1}^{N_{nuc}}\nabla_A^2 \hspace{1mm}
\tag{1.1}
\end{eqnarray}
where $m_A$ is the mass of nucleus $A$ (in kg), $e$ is the proton charge, $m_e$ is the rest mass of the electron, $Z_A$ is the atomic number (equal to the number of protons) for nucleus $A$, $\hbar= \frac{h}{2\pi}$, $r_{ij}$ is the distance between electrons $i$ and $j$, $r_{iA}$ is the distance between nucleus $A$ and electron $i$, $R_{AB}$ is the distance between nuclei $A$ and $B$, $\nabla_i^2$ is the Laplacian for electron $i$ (i.e. the sum of second partial derivatives with respect to coordinates of electron $i$), $\nabla_A^2$ is the Laplacian for nucleus $A$, and $\varepsilon_0$ is the permittivity of vacuum. The first and last terms account for the kinetic energies of the electrons and nuclei, respectively, while the electron-electron and nucleus-nucleus Coulomb repulsions are represented by third and fourth terms, respectively. Don't let the sign in front of the operators fool you! **Only** second term, which accounts for the Coulomb attraction between the nuclei and electrons, lowers the molecular energy!


As you can probably appreciate from Eq. (1.1) and also from your experience in prior chemistry classes, SI units (J) are cumbersome when discussing energy levels of atoms or molecules. Thus, to simplify notation, we introduce atomic units. In this unit system 
\begin{equation}
e = m_e = \hbar = 4\pi\varepsilon_0 = 1, 
\tag{1.2}
\end{equation}
and the molecular Hamiltonian is considerably simpler
\begin{equation}
\hat{H}_{\rm molecular} = -\frac{1}{2}\sum_{i=1}^{N_e} \nabla^2_i -\sum_{A=1}^{N_{nuc}}\sum_{i=1}^{N_e}\frac{Z_A}{r_{iA}}+\sum_{i=1}^{N_e}\sum_{j>i}^{N_e}\frac{1}{r_{ij}} + \sum_{A=1}^{N_{nuc}} \sum_{B>A}^{N_{nuc}} \frac{Z_AZ_b}{R_{AB}} -\frac{1}{2M_A}\sum_{A=1}^{N_{nuc}}\nabla_A^2 ,
\tag{1.3}
\end{equation}
where $M_A$ is the mass of nucleus $A$ in atomic units (more on this in a little bit). In atomic units, distances are measured in multiples of Bohr radii $a_0 = \frac{4\pi \varepsilon\hbar^2}{m_ee^2}=0.5292$ ${\mathring{\rm{A}}}$ and energies are measured $E_h = \frac{\hbar^2}{m_ea_0^2} = 27.2114$ eV (the atomic unit of energy IS often referred to as a Hartree, named after the English physicist Douglas Hartree). As one can conclude from  Eq. (1.2),
\begin{equation}
E_h = a_0 = 1
\end{equation}
in atomic units. Although the Hamiltonian in Eq. (1.3) appears to be slightly more manageable now, it is a second-order partial differential that depends on the coordinates of **both** the electrons **and** the nuclei. In fact, it is extremely difficult to solve the full molecular Schrödinger equation for all but the simplest molecules, and we need to make some simplifying approximation. 


Note that although the rest mass of the electron is unity in atomic units, nuclear masses are much larger; even for the lightest nucleus, hydrogen, $M_{H} = \frac{m_p}{m_e} \approx 1836$ !! This observation is in fact at the heart of the Born-Oppenheimer approximation (named after the physicists Max Born and the Robert Oppenheimer). Because of their larger mass, the positions of the nuclei can be treated as constant. If the nuclear positions $\{R_A\}$ are *fixed*, the last term in the molecular Hamiltonian vanishes, the nuclear repulsion energy (4$^{\rm th}$ term in Eq. (1.3)) 
\begin{equation}
V_{NN} = \sum_{A=1}^{N_{nuc}} \sum_{B>A}^{N_{nuc}} \frac{Z_AZ_b}{R_{AB}}
\tag{1.4}
\end{equation}
is a constant that can be added to the electronic energy $E_{el}$ (see below), and the electrons are considered to move in the electrostatic field of the nuclei. Thus, within the Born-Oppenheimer approximation, the electronic Hamiltonian is
\begin{equation}
\hat{H}_{\rm el} = \hat{T}_e + \hat{V}_{Ne} + \hat{V}_{ee} = -\frac{1}{2}\sum_{i=1}^{N_e} \nabla^2_i -\sum_{A=1}^{N_{nuc}}\sum_{i=1}^{N_e}\frac{Z_A}{r_{iA}}+\sum_{i=1}^{N_e}\sum_{j>i}^{N_e}\frac{1}{r_{ij}} 
\tag{1.5}
\end{equation}
and the electronic energy levels are obtained by solving the electronic Schrödinger equation
\begin{equation}
\hat{H}_{\rm el} \psi_{\rm el}\left(\{r_i\};\{R_A\}\right) = E_{\rm el} \psi_{\rm el}\left(\{r_i\};\{R_A\}\right)
\tag{1.6}
\end{equation}
where the notation $\psi_{\rm el}\left(\{r_i\};\{R_A\}\right)$ implies that the electronic wave function depends *explicitly* on the coordinates $\{r_i\}$ of the electrons **and** only *parametrically* on the coordinates of the nuclei (i.e. the nuclear coordinates $\{R_A\}$ do not appear in the expression, however, the form of the electronic wave function depends on where the nuclei are). Note that because the the kinetic energy of the nuclei is neglected within the Born-Oppenheimer approximation, the Born-Oppenheimer energy $E_{BO} = E_{el} + V_{NN}$ (i.e. the sum of the electronic energy and the nuclear repulsion energy) is lower than the total energy of the molecule!

In [2]:
display_quiz(bo_energy)

<IPython.core.display.Javascript object>

## I.2. Pragmatic introduction to quantum chemistry modelling

As you may already anticipate, just computing $E_{\rm el}$ itself by solving electronic Schrödinger equation in Eq. (1.6) is already of significant practical use. For example, the electronic energy of a molecule at different geometries allows the prediction of the relative energies of isomers. Similarly, one can use energies to predict the lowest-energy equilibrium geometry of molecules or to determine barrier heights and energy changes for chemical reactions. Furthermore, based on the first postulate of quantum mechanics, the electronic wave function allows us to predict any molecular property. These properties include electron densities, electronic dipole (and higher-order) moments, and NMR shielding constants. Lastly, knowing electronic energies and wave functions can collectively be used to predict the UV-Vis spectra of atoms and molecules. Although these properties are of significant practical utility, solving the electronic Schrödinger equation is still an extremely challenging task, and several approximate methods have been developed over the years. 

As with any model, the accuracy of electronic structure theory calculations is determined by the underlying approximations. Apart from the Born-Oppenheimer approximation, there are two approximations (loosely/broadly speaking) one usually makes in practical applications. The first approximation has to do with the choice of a **basis set** and how we describe the spatial distribution of individual electrons in the molecule. In other words, how accurate is our description of the molecular orbitals in terms of the atomic orbitals? The second approximation concerns our model for the interaction (repulsion) between electrons and is often referred to as **electron correlation** treatment. Ideally, we would use a very large and flexible basis set with a very accurate electron correlation treatment, but this is rarely practical as the computational time and resources needed for electronic structure calculations increase very quickly with basis set size and the level of correlation treatment (see figure below).

<img src="https://github.com/act-cms/quantum-chemistry-simulations-in-psi4/blob/main/pople-diagram.jpeg?raw=true" style="display: block; margin: 0 auto; max-height:500px;">

In [3]:
display_quiz(pople_diagram)

<IPython.core.display.Javascript object>

### I.2.1 Basis sets

The first approximation arises mainly for practical reasons. Although we have closed-form expressions for the orbitals of one-electron atoms and ions, we do not know the exact mathematical expressions for the orbitals of many-electron atoms, and our knowledge of orbitals for many-electron molecules is even less complete! Thus, to describe the distribution of electrons in molecules, molecular orbitals (MOs) are represented as linear combinations of atomic orbitals (LCAO)
\begin{equation}
\phi^{MO}_\alpha = \sum_i^{N_{AO}} c_{\alpha,i} f^{AO}_i
\tag{2.1}
\end{equation}
where the number of AOs ($N_{AO}$) is determined by the size of the molecule and the choice of basis set. While AOs describe the spatial distribution of electrons in atoms, MOs describe electron densities in molecules. As an explicit example, if we choose to only include $1s$ type AOs on the two hydrogen atoms (referred to as the *minimal* basis), the bonding $\sigma_g$ and antibonding $\sigma_u$ MOs of H$_2$ are
\begin{equation}
\sigma_g \propto 1s_A + 1s_B \hspace{3cm} \sigma_u \propto 1s_A - 1s_B
\tag{2.2}
\end{equation}
in terms of the $1s_A$ AO centered on nucleus $A$ and the $1s_B$ AO centered on nucleus $B$. Note that we ascribe a set of AOs to each atom of the molecule, and, from the example in Eq. (2.2) above, you can make the following general observations: 1) the sum over $i$ in Eq. (2.1) includes all AOs in the molecule (hence, MOs are delocalized over the entire molecule), 2) each MO is associated with its own *distinct* set of expansion coefficients, and 3) the total number of MOs is equal to the total number of AOs. Lastly, the quantity $\vert c_{\alpha,i}\vert^2$ is proportional to the importance of AO $i$ in describing MO $\alpha$; for example, both AOs are equally important in the description of the $\sigma_g$ MO.

A minimal basis set usually yields results that are *only qualitatively* correct, and we need to consider basis sets with more flexibility if we want to use electronic structure calculations *quantitatively*. For this reason, a plethora of extended basis sets have been developed over the years including split-valence basis sets, Pople-style basis sets, Karlsruhe basis sets, atomic natural orbital basis sets, and correlation-consistent basis sets. In this worksheet, we will focus on the correlation-consistent basis sets developed by Dunning and co-workers.

This sequence of basis sets allows us to systematically improve the quality of the results and extrapolate certain properties (in particular energies) to the infinite/complete basis set limit. For second-period atoms, the smallest member of this family of basis sets - the **c**orrelation-**c**onsistent **p**olarized **V**alence **D**ouble **Z**eta (cc-pVDZ) basis set - includes 3 $s$-type, 2 $p$-type and 1 $d$-type AOs. If you list the orbitals in the first three shells (up to $n=3$ and $\ell=2$) of hydrogen, you will see that this basis set includes all these orbitals. We can increase the flexibility by including an additional shell (up to $n=4$ and $\ell=3$); the correlation-consistent polarized Valence Triple Zeta AO (cc-pVTZ) basis set would thus consist of 4 $s$-type, 3 $p$-type, 2 $d$-type, and 1 $f$-type AOs. In general, these basis sets are denoted cc-pV$X$Z, where $X$ refers to the cardinal number of the basis set and, in the case of second-period atoms, is equal to the highest value of $\ell$ for the basis set.  The table below lists the different basis sets that you will encounter in the exercises below.

|  basis set  |    cardinality  <br> ($X$) |    AOs included <br> 1$^{\rm st}$ period | AOs included <br> 2$^{\rm nd}$ period |
|:---------:|:-----:|:-----------:|:-----:|
| cc-pVDZ | 2 | 2$s$,1$p$                | 3$s$,2$p$,1$d$ |
| cc-pVTZ | 3 | 3$s$,2$p$,1$d$           | 4$s$,3$p$,2$d$,1$f$ |
| cc-pVQZ | 4 | 4$s$,3$p$,2$d$,1$f$      | 5$s$,4$p$,3$d$,2$f$,1$g$ |
| cc-pV5Z | 5 | 5$s$,4$p$,3$d$,2$f$,1$g$ | 6$s$,5$p$,4$d$,3$f$,2$g$,1$h$ |

In [4]:
display_quiz(number_AOs)

<IPython.core.display.Javascript object>

### I.2.2 Electron correlation

What makes Eq. (1.5) difficult to solve is the electron-electron repulsion term in Eq. (1.4); if it wasn't for this term, the electronic Schrödinger equation would have been solved with a very high degree accuracy many years ago. Below, we will discuss three *ab initio* approximations for modeling the repulsion between electrons. An *ab initio* (also called first principles) method is one that only uses fundamental physical constants (e.g. fundamental unit of charge, etc.) and physical laws (e.g. postulates of quantum mechanics) without resorting to the use of experimental data.

<br> <center> **Hartree Fock theory** </center> <br>

The most common type of *ab initio* electronic structure approach is called the Hartree-Fock (HF) method, in which the electronic structure of a molecule is described by a single electron configuration where electrons occupy the lowest-energy orbitals, and the ground-state energy is then obtained by varying the MO expansion coefficients $\{c_{\alpha,i}\}$ in Eq. (2.1). The single electron configuration approximation is equivalent to assuming that an electron moves in the average electric field generated by the remaining $N_e-1$ electrons, where $N_e$ is the total number of electrons in the molecule. The main steps of computing the orbital expansion coefficients can be summarized as follows

1) Guess the form of the molecular orbitals.
2) Given the orbitals, construct the approximate Hamiltonian (called the Fock operator) matrix which incorporates the kinetic energy of the electrons, the nucleus-electron attractions, and the average electron-electron repulsion.
3) Diagonalize the Fock matrix to obtain orbital energies (the eigenvalues) and orbital expansion coefficients (the eigenvectors).
4) Because the Fock matrix depends on the orbitals, steps 2 and 3 are repeated until the total energy and electron density converge below user-defined thresholds.

Because the HF approximation neglects the instantaneous repulsion between electrons, the HF energy $E_{HF}$ is *above* the exact energy of the molecule. No matter how large a basis set one uses, $E_{HF} > E_{exact}$. Furthermore, also note that because most calculations use extended basis sets, the total number of MOs (denoted $N$) is greater than the number of electrons. Assuming that the molecule contains and even number of electrons, the $N_o = N_e/2$ lowest-energy MOs are doubly occupied whereas the remaining $N_v = N - N_e / 2$ MOs are unoccupied. The unoccupied MOs are sometimes referred to as virtual or external MOs. These virtual MOs may appear to be somewhat wasteful, however, their energies can be used to approximate electron affinities.

<br> <center> **Excited configurations and electron correlation** </center> <br>

More importantly, the virtual MOs can be used by more sophisticated methods to provide a quantitatively more accurate model for electron correlation. These methods are often referred to as *correlated methods* since they aim to model the effects of electron correlation by including excited configurations obtained from the lowest-energy HF configuration. To qualitatively appreciate why these configurations might result in a lower energy than $E_{HF}$, consider a minimal basis calculation for molecular hydrogen. HF theory yields two MOs (see Eq. (2.2)) with the bonding $\sigma_g$ MO doubly-occupied in the $\sigma_g^2$ lowest-energy configuration. For symmetry reasons, the only other configuration that needs to be considered is the one where the antibonding $\sigma_u$ MO is doubly occupied. At first, it may appear that this doubly-excited configuration (relative to HF) may raise the energy (since a higher-energy MO is doubly occupied), however, because the antibonding MO enhances electron density away from the internuclear region, including the $\sigma_u^2$ configuration increases the separation between electrons and hence lowers the energy!

<img src="https://github.com/act-cms/quantum-chemistry-simulations-in-psi4/blob/main/h2-minimal.jpeg?raw=true" style="display: block; margin: 0 auto; max-height:500px;">

More generally, a calculation with $N_e$ electrons, $N_o$ occupied MOs, and $N_v$ virtual (unoccupied) MOs, the exact wave function includes the HF configuration, as well as singly-, doubly-, and, in principle, $N_e$-tuply excited configurations. 

<img src="https://github.com/act-cms/quantum-chemistry-simulations-in-psi4/blob/main/excited-configurations.jpeg?raw=true" style="display: block; margin: 0 auto; max-height:500px;">

Mathematically, this means that the exact wave function is expressed as a linear combination of configurations
\begin{equation}
\psi_{exact} = \psi_{HF} + \sum_{S} c_S \psi_S + \sum_{D} c_D \psi_D + ... + \sum_{N_e} c_{N_e} \psi_{N_e} 
\tag{2.3}
\end{equation}
In Eq. (2.3), $\psi_S$ and $\psi_D$ denote singly- and doubly-excited configurations relative to the HF configuration, respectively, and the importance of a configuration is proportional to the magnitude of the coefficients $c_S$ and $c_D$. In general, the number of singly-excited configuration is proportional to $N_o\cdot N_v$ while the number of doubly-excited configurations is proportional to $N_o^2 \cdot N_v^2$. As you may anticipate, the number of configurations included in Eq. (2.3) grows rather quickly with the number of unoccupied orbitals and the excitation level! Luckily, in most cases, the higher the energy of an excited configuration, the smaller its contribution to the exact wave function, and thus, calculations including singly- and doubly-excited configurations usually yield quantitatively accurate energies and properties for most molecules near the equilibrium geometry. 

Below, we discuss three different correlated methods. In essence, these methods differ from each other in how the coefficients for the excited configurations in Eq. (2.3) are calculated, however, the details of the specific mathematics involved is beyond the scope of our discussion. We will focus our discussion on highlighting important aspects of each method as they relate to practical applications. 

<br> <center> **Møller-Plesset perturbation theory** </center> <br>

The key assumption of Møller-Plesset (MP) perturbation theory is that in most cases, the effects of electron correlation are rather small. Thus, one can use perturbation theory to estimate the coefficients of excited configurations; for example, the coefficients of doubly-excited configurations are inversely proportional to the energy difference between the HF and doubly-excited configurations. This is both the strength of the method as well as its Achilles heel; the calculations are relatively quick and do not require huge computing power as evaluating the energy differences is rather easy, however, MP perturbation theory can give egregious results when configurations become degenerate (as is often the case for transition metal complexes or for molecules far from their equilibrium geometries). Nonetheless, in the absence of such degeneracies, energies from second-order (denoted MP2) and fourth-order (denoted MP4) MP perturbation theory are readily available for relatively large molecules. As well, because of the simplicity of the mathematical expressions, the derivatives needed for geometry optimization and calculating molecular properties can be evaluated efficiently.

<br> <center> **Configuration interaction** </center> <br>

The configuration interaction (CI) method can be classified according to the maximum excitation level included in the CI wave function. If we include the HF configuration and the singly-excited configurations, we obtain the CI method with single excitations (CIS for short) that can be used to model excited states of molecules. Including singly- and doubly-excited configurations yields the CI method with single and double excitations (CISD) which can be used to incorporate correlation effects with a high level of accuracy. If all excitations are included, we obtain the full-CI method which represents the exact solution of the Schrödinger equation within the chosen AO basis set. The coefficients for excited configurations are determined using the linear variation method using iterative diagonalization methods that avoid the explicit construction of the Hamiltonian matrix. However, due to the iterative nature of the diagonalization, CI calculations are considerably more costly (both in terms of time and computing resources) than MP perturbation theory. Furthermore, except for full-CI, truncated CI methods such as CIS and CISD are not size-extensive and their accuracy degrades for larger molecules. Thus, CIS and CISD methods are usually used for medium-sized molecules in medium-sized basis sets. 

<br> <center> **Coupled cluster theory** </center> <br>

Coupled cluster (CC) theory is considered by many as the gold standard of electronic structure theory. As in CI, there exists a hierarchy of CC approximations; for example, CCSD includes single and double excitations. If we want to further improve the accuracy without incurring too great a cost penalty, the CCSD(T) method includes (perturbatively) the effect of triple excitations and provides highly accurate energies and molecular properties. However, notice that because triple excitations are included using perturbation theory, CCSD(T) can also provide nonphysical results when degeneracies occur. Although the CI and CC methods may appear similar on the surface, they are quite distinct. Because CC theory uses an exponential parameterization of the excitation operator, the CC wave function is more accurate since higher-order excited configurations are included indirectly (i.e. coefficients for quadruply excited configurations are expressed as products of coefficients of doubly- and/or singly- excited configurations). Furthermore, because the CC method is size-extensive, its accuracy does not degrade for larger molecules. Taken together, in the absence of degeneracies, *CCSD(T) represents the gold standard in quantum chemistry that is applicable to medium-sized molecules with medium-sized basis sets*.     

In [5]:
display_quiz(correlation_energy)

<IPython.core.display.Javascript object>

# II. List of acronyms and key terms used in quantum chemistry modelling

Key concepts of electronic structure theory.

- **Molecular Hamiltonian**: This operator includes the operators for the 1) kinetic energy of the electrons $\hat{T}_e$, 2) nucleus-electron potential $\hat{V}_{ne}$, 3) electron-electron potential $\hat{V}_{ee}$, 4) nucleus-nucleus potential $\hat{V}_{nn}$, and 5) kinetic energy of the nuclei $\hat{T}_n$.
\begin{equation}
\hat{H}_{\rm molecular} = \hat{T}_e + \hat{V}_{ne} + \hat{V}_{ee} + \hat{V}_{nn} + \hat{T}_n
\end{equation}

- **Born-Oppenheimer approximation**: Because the mass of a nucleus is at least 1836 times larger than that of an electron, the positions of the nuclei are assumed to be fixed. As a result, the kinetic energy of the nuclei may be neglected and the nuclear repuslion energy is a constant.

- **Electronic (aka Born-Oppenheimer) Hamiltonian**: This operator includes the operators for 1) kinetic energy of the electrons $\hat{T}_e$, 2) nucleus-electron potential $\hat{V}_{ne}$, and 3) electron-electron potential $\hat{V}_{ee}$.
\begin{equation}
\hat{H}_{\rm el} = \hat{T}_e + \hat{V}_{ne} + \hat{V}_{ee}
\end{equation}

- **atomic orbitals (AO)**: Atom-centered functions that describe the spatial distribution of a single electron in and atom.
- **molecular orbitals (MO)**: Functions that describe the spatial distribution of a single electron in a molecule.
- **linear combination of atomic orbitals (LCAO)**: Molecular orbitals are represented as a weighted sum of atomic orbitals.
- **basis set**: A set of atom-centered functions used to represent AOs in atoms and MOs in molecules.
- **infinite/complete basis set (CBS) limit**: Value of a certain property, such energy, in the limit that the basis set is complete (i.e. the basis set size approaches infinity). This value is often extrapolated as calculations with an infinite nube of basis functions are not feasible.
- **Hartree Fock (HF) theory (aka mean field theory)**: Approximate method for solving the electronic Schrödinger equation; assumes that an electron in the molecule moves in the average electric field generated by the remaining electrons (this is equivalent to the assumption that the wave function can be written as a single electron configuration).
- **electron correlation**: Refers to the idea that because of the instantaneous repulsion, the motion of electrons is correlated (i.e. the position of one electron is not independent of the instantaneous position of another electron).
- **correlated method**: A method that incorporates the effects of electron repulsion beyond HF theory
- **correlation energy**: Energy difference between the energy obtained from a correlated method and the HF energy. $E_{\rm corr} = E_{\rm correlated} - E_{\rm HF}$. Note that $E_{\rm corr} \leq 0$.
- **occupied MO**: Orbital that is occupied by 1 or 2 electrons.
- **unoccupied MO (aka virtual MO or external MO)**: Orbital that has no electrons in it.
- **singly-occupied MO**: Orbital with one electron in it.
- **boubly-occupied MO**: Orbital with two electrons in it.
- **closed-shell configuration**: A configuration with an even number of electrons *and* all MOs doubly occupied; the total spin is $S=0$.
- **open-shell configuration**: A configuration where some MOs are singly occupied; the total spin is $S > 0$.
- **singly-excited configuration**: A configuration obtained by "promoting" a single electron from an MO that is occupied in the HF configuration to an unoccupied MO. $m_s$ (spin-up or spin-down) of the promoted electron must remain unchanged.
- **doubly-excited configuration**: A configuration obtained by "promoting" two electrons from MO(s) that is/are unoccupied in the HF configuration to unoccupied MO(s). Combined $m_s$ of the promoted electrons must remain unchanged.

List of abbreviations often used in electronic structure theory:

- **AO**: atomic orbital
- **MO**: molecular orbital
- **LCAO**: linear combination of atomic orbitals
- **cc-pVDZ**: correlation-consistent polarized valence double zeta basis set 
- **cc-pVTZ**: correlation-consistent polarized valence triple zeta basis set
- **cc-pVQZ**: correlation-consistent polarized valence quadruple zeta basis set
- **cc-pV5Z**: correlation-consistent polarized valence quintuple zeta basis set
- **HF**: Hartree-Fock
- **SCF**: Self-consistent field method (in most cases, used interchangeably with HF)
- **RHF**: restricted Hartree-Fock for closed-shell configurations (spatial parts of spin-up and spin-down MOs are restricted to be the same)
- **ROHF**: restricted Hartree-Fock for open-shell configurations (spatial parts of spin-up and spin-down MOs are restricted to be the same)
- **UHF**: unrestricted Hartree-Fock (spatial parts of spin-up and spin-down MOs can be different)
- **MP2**: second-order Møller-Plesset perturbation theory
- **MP4**: fourth-order Møller-Plesset perturbation theory
- **CIS**: configuration interaction with single excitations
- **CISD**: configuration interaction with single and double excitations
- **full-CI**: configuration interaction including ALL possible excitations. The full-CI energy represents the exact energy within a given basis set.
- **CCSD**: coupled-cluster theory with single and double excitations
- **CCSD(T)**: coupled-cluster theory with single and double excitations and perturbative triples correction

# III. General information on the Psi4API interface

Psi4 is a general purpose electronic structure package that allows us to compute molecular properties using various basis sets and levels of electron correlation. Before diving in to carry out such calculations, we will briefly discuss what Psi4API is and what the user needs to input in order to perform such calculations. 

The Psi4API (Application Programming Interface) allows the seamless integration of Psi4 into Python. In this interface model, we first import the Psi4 module using

```python
import psi4
```

and then write Python code that call functions from the ```psi4``` library to set up and run the calculation of interest. 

Assuming that the ```psi4``` library has been imported, the code for running a calculation containes the following parts.

1. Molecule specification
2. Specification of parameters for calculation
3. Specification of computational resources
4. Specification of the type of calculation

Below, we will provide a short overview of each of these sections.

## III.1 Specifying the molecule

The two most common ways of specifying the geometry of a molecule are Cartesian coordinates and Z-matrices. The former are fairly intuitive since the user needs to only specify the $(x,y,z)$ coordinates for each atom. For example, assuming the molecule lies along the $z$ axis and that the carbon is at the orgin, the geometry of hydrogen cyanide in Cartesian coordinates is

```python
C  0.000 0.000  0.000
N  0.000 0.000  1.156
H  0.000 0.000 -1.064
```

While this format is intuitive, specifying the geometry for more complex (i.e. nonplanar and/or nonlinear) molecules is cumbersome in Cartesian coordinates, and we often use Z-matrix notation. Don't worry, the name sounds abstract, but you are likely familiar with most concepts involved.

In Z-matrix notation, the molecular geometry is described in terms of internal coordinates which include bond length, bond angles, and dihedral angles. A Z-matrix is built up sequentially atom-by-atom as follows:

- The first atom is specified by only its chemical symbol
- The second atom is specified by its chemical symbol, followed by the distance between it and the first atom.
- The third atom is specified by its chemical symbol, followed by the distance between it and the first atom, and the angle formed between the third, second, and first atoms.
- Any subsequent atom is specified by its chemical symbol, followed by the between it and any previously specified atom, the angle formed with two previously defined atoms, and the dihedral angle formed with three previously defined atoms.

Going back to our example of hydrogen cyanide, the Z-matrix is

```python
C
H 1 1.064
N 1 1.156 2 180.0
```

and we can use the figure below to understand how to interpret the entries in the Z-matrix. 


<img src="https://github.com/act-cms/quantum-chemistry-simulations-in-psi4/blob/main/hcn-geometry.jpeg?raw=true" style="display: block; margin: 0 auto; max-height:400px;">


The most difficult angle to visualize is the dihedral angle which corresponds to the angle between two intersecting planes. An example will best demonstrate the use of this angle. Consider the staggered conformation of ethane for which the three-dimensional geometry and corresponding Neman projection are shown in th figure below.

<img src="https://github.com/act-cms/quantum-chemistry-simulations-in-psi4/blob/main/ethane-geometry.jpeg?raw=true" style="display: block; margin: 0 auto; max-height:400px;">

The geometry can be specified in Z-matrix notation as

```python
C                            # this specifies the position of C1
C 1 1.536                    # this specifies the position of C2
H 1 1.091 2 110.91           # this specifies the position of H1
H 2 1.091 1 110.91 3  60.0   # this specifies the position of H4 which forms a dihedral angle of  60 with H4-C2-C1-H1
H 2 1.091 1 110.91 3 180.0   # this specifies the position of H5 which forms a dihedral angle of 180 with H5-C2-C1-H1
H 2 1.091 1 110.91 3 300.0   # this specifies the position of H6 which forms a dihedral angle of 300 with H6-C2-C1-H1
H 1 1.091 2 110.91 6 180.0   # this specifies the position of H2 which forms a dihedral angle of 180 with H2-C1-C2-H6
H 1 1.091 2 110.91 4 180.0   # this specifies the position of H3 which forms a dihedral angle of 180 with H3-C1-C2-H4
```

Now that we have discussed how we can specify the molecular geometry, we will discuss how to specifically use the Psi4API to specify the molecule. The basic syntax is

```python
psi4.geometry = ("""
charge spin-multiplicity
molecule geometry specification
""")
```

- <u>Charge and spin-multiplicity</u> are specified on the first line. **This information must always be provided.**
    - *charge* (integer): overall charge of the molecule (-1 for HO$^-$, +1 for NH$_4^+$, 0 for O$_2$, etc.
    - *spin-multiplicity* (integer): defines the total spin of the molecule. For the lowest-energy electronic state, the value of the spin multiplicity is $N_{\rm unp} + 1$, where $N_{\rm unp}$ is the number of unpaired electrons in the molecule. For example, the spin-multiplicity of O$_2$ is 3 while that of N$_2$ is 1.

<br>

- <u>Molecule geometry specification</u>: In this section, we can define the geometry of the molecule using Cartesian coordinates or Z-matrix notation. **This information must always be provided.**

An example, take water (neutral and no unpaired eletrons) at the experimental geometry ($\rm H-O-H$ bond angle of 104.48$^\circ$ and $\rm O-H$ bond length of 0.958 ${\mathring{\rm{A}}}$). The molecule specification within the Psi4API would be
```python
psi4.geometry = ("""
0 1
o 
h 1 0.958
h 1 0.958 2 104.48
""")
```

## III.2 Specifying the parameters for calculations

In this section, we can specify parameters for the calculations. The basic syntax is
```python
psi4.set_options({
    'keyword_1_name': keyword_1_value
    'keyword_2_name': keyword_2_value
    ...
})
```

where ```keyword_#_name``` must be a keyword recognized by Psi4 and ```keyword_#_value``` must be one of the recognized values for that keyword. Below, we will discuss some of the keywords most pertinent for our purposes. To get the full list of options, use ```psi4.core.get_global_option_list()``` or see this [website with the list of keywords for each Psi4 module](https://psicode.org/psi4manual/1.9.x/autodoc_options_c_bymodule.html). 

- basis (string): specify the basis set. Examples of recognized keyword values are `cc-pvdz`, 'cc-pvtz', 'cc-pvqz', '6-311g++'. For a full list of supported basis sets, click this [website link](https://psicode.org/psi4manual/1.9.x/basissets_tables.html#apdx-basistables).

- scf_type (string): specify the algorithm for performing th Hartree-Fock calculation. Valid keyword values include
  
    - 'pk': do not approximate the electron-electron repulsion integrals (this is the default)
    - 'df': use density fitting to approximate the electron-electron repulsion integrals
 
</br>
 
- maxiter (integer): maximum number of iterations to converge the Hartree-Fock calculation (default is 100)
      
- reference (string): specify the type of Hartree-Fock calculation. Valid keyword values are
    - 'rhf': restricted Hartree-Fock calculation (may only be used for closed-shell molecules).
    - 'rohf': restricted open-shell Hartree-Fock calculation
    - 'uhf': unrestricted Hartree-Fock calculation
 
</br>
      
- cc_type (string): specifies the type of algorithm for performing the coupled-cluster calculation. Valid keyword values include
    - 'conv': use the conventional algorithm (this is the default)
    - 'df': use the density-fitted algorithm with density-fitted electron-electron repulsion integrals

</br>

- mp_type (string): specifies the type of algorithm for performing the Møller-Plesset calculation. Valid keyword values include
    - 'conv': use the conventional algorithm (this is the default)
    - 'df': use the density-fitted algorithm with density-fitted electron-electron repulsion integrals

</br>

- freeze_core (boolean): specifies whether a core eletrons are specified. Valid keyword values are
    - True: freeze core electrons in correlated calculation (this means, $1s$ electrons are frozen for second period elements while the $1s$, $2s$, and $2p$ electrons are frozen for third period elements.
    - False: do not freeze core electrons (this is the default)

## III.3 Specifying parameters for computational resources

As discussed above, the larger the basis set and the more sophisticated the level of electron correlation treatment, the more computing resources are needed to carry out the calculation. By default, Psi4 calculations use a maximum of 256 Mb (Megabytes) of RAM. The memory can be increased using Psi4's ```psi4.set_memory()``` function. For example, ```psi4.set_memory('500 mb')``` will increase the memory to 500 megabytes while ```psi4.set_memory(' 4.0 gb')``` will increase the memory to 4.0 Gigabytes. 

Furthermore, a Psi4 calculation can be sped up by utilizing multiple threads of the CPU and running a calculation in parallel. To specify the maximum number of threads used, we can use Psi4's ```psi4.set_num_threads()``` function. To set the number of threads to 4, we would use ```psi4.set_num_threads( 4 )```. 

## III.4 Specifying the type of calculation

Once we specify the molecular geometry, the calculation parameters, and the computng resources, we are ready to do a calculation. The types of calculation we are going to use most often include calculating the energy of a molecule at some geometry (often referred to as a single-point calculation), finding the equilibrium geometry of a molecule (often called geometry optimization), and calculating the harmonic vibrational frequencies (also called frequency analysis).

The basic syntax for each of these is listed below.

- ```psi4.energy( method )``` is used to carry out a single-point calculation. The string ```method``` specifies the type of electron-correlation treatment. For a full list of recognized methods see the [Psi4 website](https://psicode.org/psi4manual/1.4.0/api/psi4.driver.energy). Below, we will note some choices for ```method```
    - 'scf': perform a Hartree-Fock calculation
    - 'ccsd': perform a CCSD calculation
    - 'ccsd(t)': perform a CCSD(T) calculation
    - 'cisd': perform a CISD calculation
    - 'mp2': perform an MP2 calculation
 
- ```psi4.optimize( method )``` is used to carry out a geometry optimization. The string ```method``` specifies the type of electron-correlation treatment. The list of recognized values for ```method``` are the sam as for ```psi4.ener()```.

- ```psi4.frequency( method )``` is used to carry out a frequency analysis. The string ```method``` specifies the type of electron-correlation treatment. The list of recognized values for ```method``` are the sam as for ```psi4.ener()```. **Note that the frequency analysis msut be performed at the equilibrium geometry.** Thus, a frequency analysis must be preceeded by a geometry optimization using the same  basis set and level of electron-correlation treatment.

# IV. Illustrative examples of quantum chemistry modelling

In [ ]:
import psi4
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spopt

## IV.1. Effects of basis set size on energy

To demonstrate the effects of basis set size, let's calculate the HF energy of the hydrogen atom using the cc-pVXZ family of basis sets. For this simple atom, we know that the exact solution is $E_{\rm exact} = -0.5$ (in atomic units). 

Note that because the ground-state electron configuration of the H atom is $1s^1$, the HF wave function is an open-shell configuration with 1 unpaired electron. As a result, the spin-multiplicity for the ground-state of the H atom is $2$ (i.e., the ground-state of hydrogen is a doublet) and we will need to either use ROHF or UHF. I personally prefer ROHF since it preserves the spin-symmetry of the wave function while UHF does not.

Now that we calculated the energies in these basis sets, we can perform an extrapolation to the infinite basis set limit. We will use a two-parameter function proposed by Helgaker and coworkers
\begin{equation}
E^X_{HF} = E_{HF}^{\infty} + A e^{-3 X}
\end{equation}
where the fitting parameter $E_{HF}^{\infty}$ is the infinite basis set limit for the Hartree-Fock energy and $X$ denotes the cardinality of the basis set (i.e. $E^2_{HF}$ corresponds to the Hartree-Fock energy in the cc-pVDZ basis set).

## IV.2. Effects of electron correlation and relative accuracy of correlated methods

To demonstrate the concept of correlation energy and the accuracy of different models for electron correlation, let's calculate the groiund-state energy of molecular nitrogen in a cc-pVDZ basis at the MP2, MP4, CISD, CCSD, and CCSD(T) levels of theory. At $R = 112.08$ pm, the exact energy (obtained from a full-CI calculation) in the cc-pVDZ basis is $E_{\rm full-CI} = -109.278338$.

## IV.3 Potential energy curve/surface scans

To demonstrate the ideas behind a potential surface/curve (PES/PEC), we will evaluate the PES for the

1. bond stretch in molecular nitrogen (SCF, CISD, and CCSD(T) levels of theory in a cc-pVDZ basis set) 
2. rotation about the double bond in ethene (SCF and MP2 level in a cc-pVDZ basis set)

Let's plot the potential energy curve for N$_2$.

Let's plot the potential energy curve for the torsion about the double bond in ethylene. The image below shows the distored (non-planar) geometry.

<img src="https://github.com/act-cms/quantum-chemistry-simulations-in-psi4/blob/main/ethylene-geometry.jpeg?raw=true" style="display: block; margin: 0 auto; max-height:400px;">

Let's plot the potentia lenergy curve to determine the barrier to rotation about the double bond.

## IV.4 Geometry optimization and vibrational frequency analysis 

From the [NIST website](https://webbook.nist.gov/chemistry/form-ser/), $R_e = 91.68$ pm and $\tilde{\nu}_e = 4138.32$ cm$^{-1}$ for H$^{19}$F. Let's see how well we can reproduce this using HF and the gold standard CCSD(T) in the cc-pVDZ, cc-pVTZ, and cc-pVQZ basis sets.

## IV.5 Reaction energetics

In the preceeding example, we focused on the accuracy of harmonic vibrational frequencies and equilibrium geometries. In this exercise, we will demonstrate how we can calculate enthalpy, entropy, and Gibbs energy changes for reactions. As an example, we take the simple reaction
\begin{equation}
\rm H_2(g) + Cl_2(g) \rightarrow 2 \hspace{1mm} HCl(g)
\end{equation}
To calculate the relevant energy changes, one first optimizes the geometries of the species involved in the reaction, followed by harmonic frequency analysis. Then, one can use the spectrocsopic constants to calculate thermodynamic corrections to the electronic energy. 
Using data from the [NIST website](https://webbook.nist.gov/chemistry/form-ser/), the experimental quantities are

- $\Delta_RH^0 = -184.6$ kJ$\cdot$mol$^{-1}$
- $\Delta_RG^0 = -190.6$ kJ$\cdot$mol$^{-1}$
- $\Delta_RS^0 =  20.04$ J$\cdot$K$^{-1}\cdot$mol$^{-1}$

Using the usual exressions for $\Delta_RH^0$ and $\Delta_R G^0$, we can calculate these values and then $\Delta_R S^0$.

## IV.6 Excited states

So far, we focused on molecules and atoms in their ground electronic states. What about the electronic excited states?

Here we will consider the three lowest-energy electronic excited levels of borane (BH). The experimental excitation energies from the [NIST website](https://webbook.nist.gov/chemistry/form-ser/) are 2.87 eV, 5.70 eV, and 6.49 eV. As was the case for the ground electronic state, there are several levels of theory that we can use. Here, we will focus on time-dependent Hartree-Fock theory (td-scf) and the equation-of-motion CC3 theory (eom-cc3, which, loosely speaking, correponds to the extension of CCSD(T) to excited states) using the aug-cc-pVTZ basis set.

## IV.7 Ionization potentials and electron affinities

As we mentioned in the introduction, one can estimate the ionization potential and electronn affinitiy of a species using the orbital energies. A better way to calculate ionization energies and electron affinitites is to actually solve for the electronic structure of the cation and anion, respectively. Below, we will do so for carbon.

Because carbon has 2 unpaired electrons in the $2p$ subshell, its ground electronic state is a triplet (spin-multiplicity of 3), while the ground electronic state of the anion (3 unpaired electrons in the $2p$ subshell) is a quartet (spin-multiplicity of 4) and the ground-electronic state of the cation with the lone electron in the $2p$ subshell is a doublet (spin-multiplicity of 2).

From the [NIST website](https://webbook.nist.gov/chemistry/form-ser/), the experimental values for the ionization energy and electron affinity are $11.26$ eV and $1.26$ eV, respectively. 